In [1]:
import numpy as np
import scipy.stats as stats

In [2]:
import lightgbm as lgbm

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders

In [4]:
from sklearn.model_selection import RandomizedSearchCV

In [5]:
predict = pd.read_csv('trocafone_kaggle_test.csv')

In [6]:
a_predecir = pd.read_csv('labels_training_set.csv')

In [7]:
processed = pd.read_csv('features_04.csv')

In [29]:
cfeats = ['mayor_evento', 'prod_visto', 'dispos',  'channel', 'region', 'opsis', 'scrnres','cuantos_leads']
for c in cfeats:
    processed[c] = processed[c].astype('category')

In [8]:
processed = processed.set_index(processed.columns[0])
processed.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,...,personas_mismo_scrnres,busco_productos,clickeo_ad,hizo_checkout,entro_desde_motor,hizo_conversion,filtro_busquedas,busco_marca,visito_sitio,hizo_lead
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,2018-05-17 12:27:47,2018-05-17 16:28:37,6,1,...,1362,False,False,True,False,False,True,False,True,False
00091926,viewed product,iPhone 6S,Computer,0,34,372,2018-05-03 22:08:29,2018-05-31 19:52:03,448,28,...,1155,False,True,True,False,False,False,True,True,False
00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,2018-03-26 14:51:11,2018-03-26 14:56:58,10,1,...,12924,False,True,False,False,False,False,True,True,False
000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,2018-05-17 11:11:45,2018-05-26 13:09:22,206,10,...,1155,False,True,True,True,True,True,True,True,False
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,2018-05-29 00:27:47,2018-05-29 00:38:07,17,1,...,12924,True,True,True,True,False,True,False,True,False


In [31]:
predict_features = processed.loc[processed.index.isin(predict['person'])].drop(['primer_visita','ultima_visita'],axis=1)

In [32]:
predict_features.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,...,personas_mismo_scrnres,busco_productos,clickeo_ad,hizo_checkout,entro_desde_motor,hizo_conversion,filtro_busquedas,busco_marca,visito_sitio,hizo_lead
person,,,,,,,,,,,,,,,,,,,,,
00091926,viewed product,iPhone 6S,Computer,0,34,372,448,28,0,16.000000,...,1155,False,True,True,False,False,False,True,True,False
00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,10,1,66,10.000000,...,12924,False,True,False,False,False,False,True,True,False
000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,206,10,5,20.600000,...,1155,False,True,True,True,True,True,True,True,False
000e4d9e,viewed product,Samsung Galaxy S6 Flat,Computer,0,13,339,411,15,1,27.400000,...,335,False,True,True,True,False,True,True,True,False
000e619d,viewed product,Motorola Moto G2 3G Dual,Computer,0,5,28,68,3,13,22.666667,...,1155,True,True,True,True,False,True,True,True,False


In [33]:
training_features = processed.loc[processed.index.isin(a_predecir['person'])]
training_features['label'] = a_predecir.set_index('person')['label']


training_labels = training_features['label']
training_features = training_features.drop(['primer_visita','ultima_visita','label'],axis=1)

C:\Users\javif\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [34]:
training_features.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,...,personas_mismo_scrnres,busco_productos,clickeo_ad,hizo_checkout,entro_desde_motor,hizo_conversion,filtro_busquedas,busco_marca,visito_sitio,hizo_lead
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,6,1,14,6.000000,...,1362,False,False,True,False,False,True,False,True,False
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,17,1,2,17.000000,...,12924,True,True,True,True,False,True,False,True,False
001802e4,ad campaign hit,iPhone 6S,Smartphone,0,1,4,19,1,0,19.000000,...,76,True,True,True,False,False,True,False,True,False
0019e639,viewed product,Motorola Moto G2 3G Dual,Computer,2,19,189,471,115,9,4.095652,...,8873,True,True,True,True,True,True,True,True,False
001b0bf9,viewed product,Samsung Galaxy S8 Plus,Ninguno,1,0,2,7,1,9,7.000000,...,588,False,False,True,False,True,True,True,False,False


In [44]:
params = {
    'num_leaves':stats.randint(90,5000),
    'min_data_in_leaf':stats.randint(100,1000),
    'max_bin':stats.randint(70,300),
    'learning_rate':stats.uniform(0.01,1-0.01),
    'n_estimators':stats.randint(150,500)
}
grid = RandomizedSearchCV(lgbm.LGBMClassifier(boosting_type='dart'),
                          param_distributions=params,
                          scoring='roc_auc',
                          cv=2,
                          verbose=1,
                          n_iter=150)

In [45]:
xtr,xte,ytr,yte=train_test_split(training_features,training_labels)

In [46]:
grid.fit(xtr,ytr)

Fitting 2 folds for each of 150 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  7.3min finished


RandomizedSearchCV(cv=2, error_score='raise',
          estimator=LGBMClassifier(boosting_type='dart', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
          fit_params=None, iid=True, n_iter=150, n_jobs=1,
          param_distributions={'num_leaves': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000017C31821940>, 'min_data_in_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000017C31821668>, 'max_bin': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000017C31821C50>, 'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000017C312D6A20>, 'n_estimators': <scipy.stats._distn_infrastructure.rv

In [47]:
grid.score(X=xte,y=yte)

0.8403253062080364

In [48]:
grid.best_params_

{'learning_rate': 0.06748371705664348,
 'max_bin': 205,
 'min_data_in_leaf': 481,
 'n_estimators': 187,
 'num_leaves': 956}

## Prediccion

In [49]:
predictor = grid.best_estimator_

In [50]:
predictor.feature_importances_

array([  3, 299,   0, 150,  62, 253, 571, 152, 595, 378,   0,   0,  81,
         0, 398,  70,  51,  55,  78,  59, 121,   1,   0, 161, 185,  14,
         1, 192, 107,   0,  13,  79,   0,   0])

In [51]:
predictor.fit(training_features,training_labels)

LGBMClassifier(boosting_type='dart', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.06748371705664348,
        max_bin=205, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_data_in_leaf=481, min_split_gain=0.0,
        n_estimators=187, n_jobs=-1, num_leaves=956, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [53]:
prediction_submit = predictor.predict_proba(predict_features)[:,1]

In [54]:
predict_features['label']=prediction_submit

In [55]:
submit = predict.set_index('person')

In [56]:
submit['label'] = predict_features['label']

In [57]:
submit.to_csv('submit.csv')